## Statistical tests

In [1]:
import pandas as pd
import reframed

from matplotlib import colors
import sys
sys.path.append("../functions_steadiercom/")
sys.path.append("../functions/")
import supplementary_process_data as process_data
import supplementary_statistics
import general_functions as general_func

import steadiercom_samples_preprocessing as steadiercom_pre


In [2]:
%load_ext autoreload
%autoreload 2

In [61]:
communityid2name_dict = {"CD_A":"Compost digestate - Avicel",
"CD_P":"Compost digestate - PASC",
"CD_X":"Compost digestate - Xylan",
"CM_A":"Cow manure - Avicel",
"CM_P":"Cow manure - PASC",
"CM_X":"Cow manure - Xylan",
"M_P":"Marshland soil - PASC",
"M_X":"Marshland soil - Xylan",
}


SC1_C = pd.read_csv("../output_30_08_24/steadiercom_sample_0.1.3/results/results_99_SC1_C.tsv",sep="\t")
SC2_C = pd.read_csv("../output_30_08_24/steadiercom_sample_0.1.3/results/results_99_SC2_C.tsv",sep="\t")
SC1_X = pd.read_csv("../output_30_08_24/steadiercom_sample_0.1.3/results/results_99_SC1_X.tsv",sep="\t")

steadier_sample = pd.concat([SC1_C,SC2_C,SC1_X])


steadier_sample = steadier_sample[(steadier_sample.rate>1e-6)].copy()
steadier_sample = steadier_sample[(steadier_sample.frequency>0.1)].copy()
steadier_sample.mass_rate = steadier_sample.mass_rate*1000

In [62]:
all_mags_paper = general_func.read_allmags_data()
all_mags_paper_reduced,total_members_genus = general_func.all_mags_paper_genus(all_mags_paper,prefix=True)
all_mags_paper_reduced,total_members_family = general_func.all_mags_paper_family(all_mags_paper_reduced,prefix=True,combine=True)

In [63]:
steadier_sample = process_data.process_data(steadier_sample,all_mags_paper_reduced=all_mags_paper_reduced)

In [65]:
steadier_sample_cross = steadier_sample[(steadier_sample.donor!="environment") & (steadier_sample.receiver!="environment")]
steadier_sample_cross

,donor,receiver,compound,mass_rate,rate,frequency,community,medium,mass_rate*frequency,super_class,genus_donor,genus_receiver,family_donor,family_receiver
13,CH15-bin.0,CH15-bin.7,D-Glucose,4.921516,0.027318,0.27,CD_P,SC1_C,1.328809,simple sugars,g_DUOS01,g_Sphaerochaeta,f_Treponemataceae,f_Sphaerochaetaceae
33,CH13-bin.11,CH13-bin.4,Cellobiose,2.822338,0.008245,0.11,CD_A,SC1_C,0.310457,oligosaccharides,g_Desulfovibrio,g_Bacteroides,f_Desulfovibrionaceae,f_Bacteroidaceae
36,CH13-bin.14,CH13-bin.25,D-Glucose,2.705991,0.015020,0.20,CD_A,SC1_C,0.541198,simple sugars,g_Lacrimispora,g_Sphaerochaeta,f_Lachnospiraceae,f_Sphaerochaetaceae
39,CH15-bin.7,CH15-bin.8,D-Glucose,2.435377,0.013518,0.12,CD_P,SC1_C,0.292245,simple sugars,g_Sphaerochaeta,g_Desulfocurvibacter,f_Sphaerochaetaceae,f_Desulfovibrionaceae
41,CH15-bin.23,CH15-bin.8,D-Glucose,2.385105,0.013239,0.17,CD_P,SC1_C,0.405468,simple sugars,g_DTFZ01,g_Desulfocurvibacter,Other,f_Desulfovibrionaceae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,CH3-bin.2,CH3-bin.0,Thymine,0.000461,0.000004,0.33,M_X,SC1_X,0.000152,nucleotides and derivatives,g_Clostridium,g_Sporolactobacillus,f_Clostridiaceae,Other
347,CH14-bin.1,CH14-bin.4,Hydrogen,0.000415,0.000206,0.18,CD_X,SC1_X,0.000075,gases,g_Lacrimispora,g_Desulfovibrio,f_Lachnospiraceae,f_Desulfovibrionaceae
351,CH14-bin.4,CH14-bin.2,Hydrogen sulfide,0.000274,0.000008,0.73,CD_X,SC1_X,0.000200,gases,g_Desulfovibrio,g_Robinsoniella,f_Desulfovibrionaceae,f_Lachnospiraceae
353,CH9-bin.2,CH9-bin.5,Hydrogen sulfide,0.000254,0.000007,0.22,CM_X,SC1_X,0.000056,gases,g_Clostridium,g_Lentilactobacillus,f_Clostridiaceae,f_Lactobacillaceae


### Interactions - Statistics

In [66]:
family_groups,mag2family_dict = general_func.mag2family(all_mags_paper_reduced)

In [67]:
pd.Series({family:len(mags) for family,mags in family_groups.items()})

Other                    19
f_Bacteroidaceae          4
f_CAG-74                  2
f_Clostridiaceae          3
f_Desulfobulbaceae        2
f_Desulfovibrionaceae    12
f_Dysgonomonadaceae       4
f_Fibrobacteraceae        2
f_Lachnospiraceae         8
f_Lactobacillaceae        2
f_Lentimicrobiaceae       2
f_Sphaerochaetaceae       6
f_Treponemataceae         4
f_Verruco-01              3
dtype: int64

In [68]:
pd.Series({family:len(mags) for family,mags in family_groups.items()}).sum()

73

In [69]:
all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","Genus","new_coverage"]].sort_values(["Source","Substrate"])

,Source,Substrate,Family,Genus,new_coverage
MAG,,,,,
CH13-bin.12,Compost_Digestate,Avicel,f_Fibrobacteraceae,g_Fibro-01,36.526676
CH13-bin.11,Compost_Digestate,Avicel,f_Desulfovibrionaceae,g_Desulfovibrio,10.155790
CH13-bin.4,Compost_Digestate,Avicel,f_Bacteroidaceae,g_Bacteroides,10.043970
CH15-bin.0,Compost_Digestate,PASC,f_Treponemataceae,g_DUOS01,23.919040
CH15-bin.23,Compost_Digestate,PASC,Other,g_DTFZ01,10.319866
CH14-bin.1,Compost_Digestate,Xylan,f_Lachnospiraceae,g_Lacrimispora,54.707294
CH14-bin.2,Compost_Digestate,Xylan,f_Lachnospiraceae,g_Robinsoniella,18.831114
CH14-bin.0,Compost_Digestate,Xylan,Other,g_Caproiciproducens,15.617491
CH8-bin.22,Cow_Manure,Avicel,f_Fibrobacteraceae,g_Fibro-01,42.725525


### All communities

#### Likelyhood that family X is receiving compound A

In [70]:
statistics_df = supplementary_statistics.statistics_function(steadier_sample_cross,"family_receiver","compound",family_groups=family_groups,metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = supplementary_statistics.statistics_adjustments(statistics_df)
statistics_df.shape

(205, 7)

In [71]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,compound,,,,,,,
f_Lactobacillaceae,L-Ornithine,0.000104,inf,"[[2.0, 0.0], [0.0, 71.0]]",2.0,0,1.0,0.021409


#### Likelyhood that family X is donating a compound A to another family

In [72]:
statistics_df = supplementary_statistics.statistics_function(steadier_sample_cross,"family_donor","compound",family_groups=family_groups,metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = supplementary_statistics.statistics_adjustments(statistics_df)
statistics_df.shape

(172, 7)

In [73]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

p_value odds_ratio  \
family_donor        compound                                     
f_Fibrobacteraceae  Indole                  0.00176      17.75   
f_Lachnospiraceae   Guanine                0.000903   6.770833   
f_Lactobacillaceae  L-Ornithine            0.000104        inf   
f_Sphaerochaetaceae D-Glucose              0.000501   5.583333   
                    D-Mannose 1-phosphate  0.000197  14.888889   

                                                                 data  \
family_donor        compound                                            
f_Fibrobacteraceae  Indole                  [[2.0, 0.0], [4.0, 67.0]]   
f_Lachnospiraceae   Guanine                 [[5.0, 3.0], [6.0, 59.0]]   
f_Lactobacillaceae  L-Ornithine             [[2.0, 0.0], [0.0, 71.0]]   
f_Sphaerochaetaceae D-Glucose              [[5.0, 1.0], [10.0, 57.0]]   
                    D-Mannose 1-phosphate   [[4.0, 2.0], [3.0, 64.0]]   

                                          # interaction # no interaction    i  \
family_donor        compound                                                    
f_Fibrobacteraceae  Indole                          2.0                0  7.0   
f_Lachnospiraceae   Guanine                         5.0                3  6.0   
f_Lactobacillaceae  L-Ornithine                     2.0                0  1.0   
f_Sphaerochaetaceae D-Glucose                       5.0                1  4.0   
                    D-Mannose 1-phosphate           4.0                2  2.0   

                                           p_value_benjamini_h  
family_donor        compound                                    
f_Fibrobacteraceae  Indole                            0.043238  
f_Lachnospiraceae   Guanine                           0.025897  
f_Lactobacillaceae  L-Ornithine                       0.017962  
f_Sphaerochaetaceae D-Glucose                         0.021544  
                    D-Mannose 1-phosphate             0.016979

#### Likelyhood that family X is receiving from family Y

In [75]:
statistics_df = supplementary_statistics.statistics_function(steadier_sample_cross,"family_receiver","family_donor",
                                                             family_groups=family_groups,metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = supplementary_statistics.statistics_adjustments(statistics_df)
statistics_df.shape

(93, 7)

In [76]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,family_donor,,,,,,,


### Cellulose communities

In [77]:
all_mags_paper_reduced2 = all_mags_paper_reduced[(all_mags_paper_reduced.Substrate=="PASC")|(all_mags_paper_reduced.Substrate=="Avicel")].copy()
family_groups,mag2family_dict = general_func.mag2family(all_mags_paper_reduced2)

In [78]:
steadier_sample_cross_cell = steadier_sample_cross[(steadier_sample_cross.community.str.endswith("_A")) | (steadier_sample_cross.community.str.endswith("_P"))]

#### Likelyhood that family X is receiving compound A

In [79]:
statistics_df = supplementary_statistics.statistics_function(steadier_sample_cross_cell,"family_receiver","compound",family_groups=family_groups,metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = supplementary_statistics.statistics_adjustments(statistics_df)
statistics_df.shape

(170, 7)

In [82]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,compound,,,,,,,
f_Lachnospiraceae,Thymine,0.000199,5.0,"[[5.0, 0.0], [11.0, 44.0]]",5.0,0,1.0,0.03382


#### Likelyhood that family X is donating a compound A to another family

In [87]:
statistics_df = supplementary_statistics.statistics_function(steadier_sample_cross_cell,"family_donor","compound",family_groups=family_groups,metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = supplementary_statistics.statistics_adjustments(statistics_df)
statistics_df.shape

(145, 7)

In [88]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_donor,compound,,,,,,,


#### Likelyhood that family X is receiving from family Y

In [89]:
statistics_df = supplementary_statistics.statistics_function(steadier_sample_cross_cell,"family_receiver","family_donor",family_groups=family_groups,metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = supplementary_statistics.statistics_adjustments(statistics_df)
statistics_df.shape

(73, 7)

In [90]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,family_donor,,,,,,,
